## load 参数

In [1]:
import numpy as np
import cv2
import scipy.io as scio

Params=scio.loadmat('workspace.mat')
K = np.array(Params['K'])
K = K.T
distortion=np.array(Params['distortion'])

## sift特征匹配

In [2]:
import numpy as np
import cv2
from matplotlib import pyplot as plt

imgpath = 'images/'
MIN_MATCH_COUNT = 10

def match_pts(imgname1, imgname2):
    img1 = cv2.imread(imgpath+imgname1, 0)          # queryImage
    img2 = cv2.imread(imgpath+imgname2, 0)        # trainImage

    # Initiate SIFT detector
    sift = cv2.xfeatures2d.SIFT_create()

    # find the keypoints and descriptors with SIFT
    kp1, des1 = sift.detectAndCompute(img1,None)
    kp2, des2 = sift.detectAndCompute(img2,None)

    FLANN_INDEX_KDTREE = 0
    index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
    search_params = dict(checks = 50)

    flann = cv2.FlannBasedMatcher(index_params, search_params)

    matches = flann.knnMatch(des1,des2,k=2)

    # store all the good matches as per Lowe's ratio test.
    good = []
    pts1 = []
    pts2 = []
    
    # ratio test as per Lowe's paper
    for i,(m,n) in enumerate(matches):
        if m.distance < 0.8*n.distance:
            good.append(m)
            pts2.append(kp2[m.trainIdx].pt)
            pts1.append(kp1[m.queryIdx].pt)

    pts1 = np.int32(pts1)
    pts2 = np.int32(pts2)
    # return the np.array type
    return pts1, pts2

# estimate估计参数

In [3]:
from numpy import *
from scipy import *
from scipy.linalg import *
from pylab import *
import cv2
import sys

def find_fundamental(pts1, pts2):
    F, mask = cv2.findFundamentalMat(pts1,pts2,cv2.FM_LMEDS)
    return F
    
def find_essential(F, K1, K2):
    return K2.T @ F @ K1

def skew(a):
    #Skew matrix A such that cross(a,v) = Av for any v
    return array([[0,-a[2],a[1]],[a[2],0,-a[0]],[-a[1],a[0],0]])

def anti_skew(A):
    return array([A[2,1], A[2,0], A[1,0]])

def triangulate(P1, P2, pts1, pts2):
    n1 = pts1.shape[0]
    n2 = pts2.shape[0]
    
    p1 = vstack([ pts1.T, ones(n1) ]).T
    p2 = vstack([ pts2.T, ones(n2) ]).T
    
    P = zeros((n1, 4))
    for i in range(n1):
        cross1 = skew(p1[i])
        cross2 = skew(p2[i])
        Q = vstack([cross1 @ P1, cross2 @ P2])
        U,S,VT = svd(Q)
        z = VT[-1, :]
        P[i, :] = z/z[3]
    
    return P[:, 0:3]

def guess_rotation(E):
    # make sure E is rank 2
    U,S,V = svd(E)
    if det(dot(U,V)) < 0:
        V = -V
    E = dot(U,dot(diag([1,1,0]),V))    

    W = array([[0,-1,0],[1,0,0],[0,0,1]])
        
    R21 = dot(U,dot(W,V))
    R22 = dot(U,dot(W.T,V))
    
    T21 = anti_skew(E @ R21.T) 
    T22 = anti_skew(E @ R22.T)
    
    return R21, T21, R22, T22

def test_rotation(P1, P2, pts1, pts2):
    #temp = cv2.triangulatePoints(P1, P2, transpose(pts1), transpose(pts2))
    temp = triangulate(P1, P2, pts1, pts2)
    for v in temp:
        if v[-1] < 0:
            return False
    return True

def errnum_rotation(P1, P2, pts1, pts2):
    #temp = cv2.triangulatePoints(P1, P2, transpose(pts1), transpose(pts2))
    temp = triangulate(P1, P2, pts1, pts2)
    num = 0
    for v in temp:
        if v[-1] < 0:
            num += 1
    return num
    

def find_rotation(E, K1, K2, pts1, pts2):
    R1 = eye(3)
    T1 = array([0, 0, 0])
    P1 = K1 @ vstack([R1.T, T1]).T
    
    R21, T21, R22, T22 = guess_rotation(E)
    
    P21 = K2 @ vstack([R21.T, T21]).T
    P22 = K2 @ vstack([R22.T, T22]).T

    #key1 = test_rotation(P1, P21, pts1, pts2)
    #key2 = test_rotation(P1, P22, pts1, pts2)
    
    
    errn1 = errnum_rotation(P1, P21, pts1, pts2)
    errn2 = errnum_rotation(P1, P22, pts1, pts2)
    
    R2 = R21
    T2 = T21

    if T21[0] < 0:
        R2 = R21
        T2 = T21
    elif T22[0] < 0:
        R2 = R22
        T2 = T22
    else:
        print('Can not find the right R matrix.')
        sys.exit()

    return R1, T1, R2, T2
    
def find_pose(R1, T1, R2, T2, K1, K2):
    #return K @ vstack([R.T, T]).T
    P1 = K1 @ vstack([R1.T, T1]).T
    P2 = K2 @ vstack([R2.T, T2]).T
    
    return P1, P2


## 运行

In [4]:
pts1, pts2 = match_pts('1_.jpg', '2_.jpg')
print(pts1.shape)
F = find_fundamental(pts1, pts2)
E = find_essential(F, K, K)
R1, T1, R2, T2 = find_rotation(E, K, K, pts1, pts2)

print(R1, T1, R2, T2, sep='\n')
K1 = K
K2 = K
P1, P2 = find_pose(R1, T1, R2, T2, K1, K2)
points = triangulate(P1, P2, pts1, pts2)

(1340, 2)
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
[0 0 0]
[[ 0.99680378 -0.01337628  0.07876103]
 [ 0.01358739  0.99990539 -0.00214501]
 [-0.07872489  0.00320831  0.99689122]]
[-0.9751416   0.00491469  0.22152812]


## MVS多图

In [5]:
def sparse(imgname1, imgname2, K1, K2):
    pts1, pts2 = match_pts(imgname1, imgname2)
    F = find_fundamental(pts1, pts2)
    E = find_essential(F, K1, K2)
    
    R1, T1, R2, T2 = find_rotation(E, K1, K2, pts1, pts2)
    
    P1, P2 = find_pose(R1, T1, R2, T2, K1, K2)
    
    X = triangulate(P1, P2, pts1, pts2)
    R = R2
    T = T2
    
    return X, R, T

X, R, T = sparse('1_.jpg', '2_.jpg', K, K)
print(type(X))

<class 'numpy.ndarray'>


In [6]:
zz = 0
ff = 0
for i in range(len(X)):
    if X[i][2] < 0:
        ff += 1
    else:
        zz += 1
print(ff, zz)

68 1272


## 多图融合

In [7]:
def multi_sparse(imgs, K):
    n = len(imgs)
    xn = n-1
    Xs = []
    Rs = []
    Ts = []
    for i in range(xn):
        Xtemp, Rtemp, Ttemp = sparse(imgs[i], imgs[i+1], K, K)
        print(i, Xtemp[10:20,])
        print(Rtemp, Ttemp)
        
        Xs.append(Xtemp)
        Rs.append(Rtemp)
        Ts.append(Ttemp)
    
    X = Xs[0]

    for i in range(1, xn):
        newX = Xs[i]
        nn = newX.shape[0]
        newX = newX.T
        for j in range(1, i + 1):
            newX = vstack([Rs[i-j].T, Ts[i-j]]).T @ vstack([newX, ones(nn)])
        newX = newX.T
        X = vstack([X, newX])
        
    return X
    
imgs = ['1_.jpg', '2_.jpg', '3_.jpg', '4_.jpg', '5_.jpg', '6_.jpg']
X = multi_sparse(imgs[:4], K)
print(X.shape)

0 [[ -8.39558971 -10.27431466  22.13363896]
 [ -5.13965252  -5.21712877  14.02206751]
 [ -4.82951437   5.75686926  13.37125398]
 [ -5.75668715  -4.85081324  15.05946156]
 [ -4.90298465  -4.55609082  13.48946696]
 [ -4.97850451   6.05451593  13.86310917]
 [ -5.14078813   2.52998535  14.35758407]
 [ -3.20902878   2.58384161   9.0981869 ]
 [ -4.08513605   1.72876616  11.5925245 ]
 [ -5.2637713   -5.91326255  14.88697377]]
[[ 0.99680378 -0.01337628  0.07876103]
 [ 0.01358739  0.99990539 -0.00214501]
 [-0.07872489  0.00320831  0.99689122]] [-0.9751416   0.00491469  0.22152812]
1 [[ 0.05964713  0.52510378 -1.38823431]
 [-0.31172133  0.46148397  1.37208549]
 [-0.31201987  0.46078908  1.37365868]
 [-0.44849958  0.24618113  1.97774249]
 [ 0.05417247  0.13777421  0.17497501]
 [ 0.68216911  0.87908251 -2.23902287]
 [ 0.98210732  2.09423169 -4.41921922]
 [ 0.70410677  0.44162732 -2.90240041]
 [ 0.63685409  0.80797834 -2.08863421]
 [ 0.69870712  0.43093403 -2.89541509]]
[[ 0.92265241  0.35112124  0

In [8]:
print(X[1500:1510,], X[1800:1810,], X[3020:3030, ], sep='\n*****')

[[-0.43200328 -0.02935312 -2.62225017]
 [-0.7042851  -0.01103257 -1.84742333]
 [-0.68783257 -0.01168466 -1.84854386]
 [-1.11268788 -1.05320654 -1.83454405]
 [-0.50639417  0.18318661 -2.64628221]
 [-0.47857243 -2.45043829  1.25741248]
 [-0.20863968  1.23486396 -4.0034544 ]
 [-0.24064241  0.91499464 -3.64221586]
 [-0.32965681  0.28872281 -3.01313037]
 [-1.26072485 -1.03251414 -1.9034581 ]]
*****[[-2.16473343  7.39462783 -9.3044365 ]
 [-2.4568916   0.01109327 -4.45500386]
 [-2.07979173 -0.50729613 -2.55496845]
 [-1.90994677 -0.17060633 -2.75879771]
 [-1.91416087  1.33033592 -3.93944817]
 [-1.96627216  1.85508184 -4.47578521]
 [-1.96198943  0.19031885 -2.43652036]
 [-1.95428361  0.96184869 -3.73680131]
 [-1.54242904 -0.26856018 -2.36429069]
 [-2.00618626  2.03445632 -4.67768973]]
*****[]


In [9]:
print(F)

[[ 1.75403911e-07  1.56012202e-05 -1.11554963e-02]
 [-9.83158864e-06  2.27223274e-07 -5.96975343e-02]
 [ 7.55013245e-03  5.70905777e-02  1.00000000e+00]]


In [10]:
Farr = array(F)
print(Farr)
print(trace(Farr.T @ Farr))

[[ 1.75403911e-07  1.56012202e-05 -1.11554963e-02]
 [-9.83158864e-06  2.27223274e-07 -5.96975343e-02]
 [ 7.55013245e-03  5.70905777e-02  1.00000000e+00]]
1.0070045796023517


In [11]:
Earr = array(E)
print(Earr)
print(trace(Earr.T @ Earr))

[[  0.16376032  14.54592283   0.2322583 ]
 [ -9.16656054   0.21156744 -62.98278073]
 [  0.51749039  64.03419144  -0.37492419]]
8363.351932203246
